In [ ]:
import pandas as pd 
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import ast

In [ ]:
# Reading and loading datasets
movie_df = pd.read_csv('movies.csv')

movie_df = movie_df.sample(frac=0.5, random_state=42)

# movie_df.to_csv('movies_half.csv', index=False)

movie_df.info()

In [ ]:
movie_df['genres'] = movie_df['genres'].apply(lambda x: x.split('-')[0] if isinstance(x, str) else x)

movie_df.to_csv('modified_file.csv', index=False)